In [3]:
import os
import re
import xml.etree.ElementTree as ET
import sumolib
import pandas as pd
from IPython.display import display
import sys

sys.path.append('/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/src') 
from config import *

In [4]:
from sumolib.net import readNet

net = readNet("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-repairment/cleaned_p2_2_newtest-osm.net.xml")

tls = [j for j in net.getNodes() if j.getType() == "traffic_light"]

print("Number of junctions controlled by traffic signals:", len(tls))


Number of junctions controlled by traffic signals: 24821


In [5]:
import xml.etree.ElementTree as ET

tree = ET.parse("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-repairment/cleaned_p2_2_newtest-osm.net.xml")
root = tree.getroot()

for conn in list(root.findall("connection")):
    if conn.get("state") == "blocked_by_restrictions":
        root.remove(conn)

for tr in list(root.findall("turnRestriction")):
    root.remove(tr)

tree.write("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-repairment/cleaned_p2_3_newtest-osm.net.xml")

In [6]:
import sumolib

net = sumolib.net.readNet("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/network/newtest-osm.net.xml")
for edge in net.getEdges():
    if "turcon" in edge.getName().lower():
        print(f"Edge: {edge.getID()}, From: {edge.getFromNode().getID()} → To: {edge.getToNode().getID()}, Allowed: {edge.allows('passenger')}, Is one-way: {not edge.allows('passenger') or edge.getLaneNumber() == 1}")


Edge: -124060635, From: 33806164 → To: 1381709067, Allowed: True, Is one-way: True
Edge: -150925676, From: 1637868895 → To: 348504234, Allowed: True, Is one-way: True
Edge: 124060635, From: 1381709067 → To: 33806164, Allowed: True, Is one-way: True
Edge: 150925674, From: 348504234 → To: 33806164, Allowed: True, Is one-way: True
Edge: 150925676, From: 348504234 → To: 1637868895, Allowed: True, Is one-way: True


In [4]:
!netconvert --help

Eclipse SUMO netconvert Version v1_22_0+0959-576b58424eb
 Copyright (C) 2001-2025 German Aerospace Center (DLR) and others; https://sumo.dlr.de
Network importer / builder for the microscopic, multi-modal traffic simulation
 SUMO.

Usage: netconvert [OPTION]*
Configuration Options:
  -c, --configuration-file FILE        Loads the named config on startup
  -C, --save-configuration FILE        Saves current configuration into FILE
  --save-configuration.relative        Enforce relative paths when saving the
                                        configuration
  --save-template FILE                 Saves a configuration template (empty)
                                        into FILE
  --save-schema FILE                   Saves the configuration schema into FILE
  --save-commented                     Adds comments to saved template,
                                        configuration, or schema

Input Options:
  -s, --sumo-net-file FILE             Read SUMO-net from FILE
  -n, --node

In [ ]:
import re
import sumolib

NET_PATH = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-repairment/cleaned_p2_2_newtest-osm.net.xml"
LOG_PATH = "/path/to/duarouter.log"

net = sumolib.net.readNet(NET_PATH)

# Regex để tìm lỗi "No connection between edge 'a' and edge 'b'"
pattern = re.compile(r"No connection between edge '([^']+)' and edge '([^']+)'")

# Tìm cặp lỗi đầu tiên có thể kiểm tra được
with open(LOG_PATH, encoding="utf-8", errors="ignore") as f:
    for line in f:
        match = pattern.search(line)
        if not match:
            continue
        from_id, to_id = match.groups()

        if not net.hasEdge(from_id) or not net.hasEdge(to_id):
            continue

        from_edge = net.getEdge(from_id)
        to_edge = net.getEdge(to_id)

        from_node = from_edge.getToNode()
        to_node = to_edge.getFromNode()

        print("❌ Route not found:")
        print(f"  FROM edge: {from_id} → toNode = {from_node.getID()}")
        print(f"    - numLanes = {from_edge.getLaneNumber()}, speed = {from_edge.getSpeed()}, isForward = {not from_id.startswith('-')}")
        print(f"  TO edge:   {to_id} ← fromNode = {to_node.getID()}")
        print(f"    - numLanes = {to_edge.getLaneNumber()}, speed = {to_edge.getSpeed()}, isForward = {not to_id.startswith('-')}")
        print(f"👉 Same node? {'✅' if from_node.getID() == to_node.getID() else '❌'}")
        break  # chỉ lấy 1 ví dụ đầu tiên để kiểm tra
